In [1]:
# 安裝必要套件
%pip install --upgrade tensorflow yfinance pandas numpy matplotlib scikit-learn --quiet

# 匯入套件
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

# 1. 載入 NVDA 歷史股價資料
df = yf.download('NVDA', start='2024-01-01')
df = df[['Close']].dropna()

# 2. 正規化資料
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(df)

# 3. 製作訓練資料（60 天窗口）
X, y = [], []
for i in range(60, len(scaled_data)):
    X.append(scaled_data[i-60:i, 0])
    y.append(scaled_data[i, 0])
X, y = np.array(X), np.array(y)
X = np.reshape(X, (X.shape[0], X.shape[1], 1))

# 4. 建構 LSTM 模型
model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(X.shape[1], 1)),
    Dropout(0.2),
    LSTM(50),
    Dropout(0.2),
    Dense(1)
])
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X, y, epochs=20, batch_size=32, verbose=1)

# 5. 預測從今天到 8 月底
future_days = (pd.Timestamp('2025-08-31') - df.index[-1]).days
predicted_prices = []

last_60_days = scaled_data[-60:].reshape(1, 60, 1)
for _ in range(future_days):
    pred = model.predict(last_60_days, verbose=0)
    predicted_prices.append(pred[0][0])
    last_60_days = np.append(last_60_days[:, 1:, :], [[pred]], axis=1)

# 6. 還原價格
predicted_prices = scaler.inverse_transform(np.array(predicted_prices).reshape(-1, 1))
forecast_dates = pd.date_range(start=df.index[-1] + pd.Timedelta(days=1), periods=future_days)

# 7. 畫出圖表
plt.figure(figsize=(14, 6))
plt.plot(df.index, df['Close'], label='Historical Close Price')
plt.plot(forecast_dates, predicted_prices, label='LSTM Forecast (to Aug 31)', linestyle='--')
plt.title('NVIDIA Stock Price Forecast (LSTM)')
plt.xlabel('Date')
plt.ylabel('Price (USD)')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


Note: you may need to restart the kernel to use updated packages.


ModuleNotFoundError: No module named 'imp'